<a href="https://colab.research.google.com/github/Rushi908/Langchain/blob/main/Self_Query_Reteriver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain-community accelerate PyPDF2 faiss-cpu langchain_chroma langchain-huggingface

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from langchain_community.llms import HuggingFacePipeline
llm=HuggingFacePipeline.from_model_id(
                                  model_id="meta-llama/Llama-3.2-1B",
                                  task="text-generation",
                                  pipeline_kwargs=dict(
                                    max_new_tokens=512,
                                    do_sample=True,
                                    repetition_penalty=1.03,
                                    return_full_text=False,
                                      ),
                                    device=0
                                    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

# Change 'model_field' to 'model_name'
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-4-d57fca443f44>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="A hacker discovers reality is a simulation and leads a rebellion against the machines controlling it.",
        metadata={"year": 1999, "director": "Lana Wachowski, Lilly Wachowski", "rating": 8.7, "genre": "science fiction"},
    ),
    Document(
        page_content="A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.",
        metadata={"year": 1994, "rating": 8.5, "genre": "animated"},
    ),
    Document(
        page_content="Batman faces off against the Joker, a criminal mastermind who plunges Gotham into chaos.",
        metadata={"year": 2008, "director": "Christopher Nolan", "rating": 9.0, "genre": "action"},
    ),
    Document(
        page_content="A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.",
        metadata={"year": 2014, "director": "Christopher Nolan", "rating": 8.6, "genre": "science fiction"},
    )
]



In [6]:
vectorstores=Chroma.from_documents(docs, embedding)

In [7]:
question1 = "Which 1994 animated movie has a rating of 8.5?"

In [8]:
vectorstores.similarity_search(question1)

[Document(metadata={'genre': 'animated', 'year': 1995}, page_content='Toys come alive and have a blast doing so'),
 Document(metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'),
 Document(metadata={'genre': 'animated', 'rating': 8.5, 'year': 1994}, page_content='A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.')]

In [9]:
reteriver=vectorstores.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [10]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough,RunnableLambda
from langchain.schema.output_parser import StrOutputParser

In [11]:
import textwrap
def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [12]:
template="""answer the Question based only the following context:
{context}

Question: {question}"""


In [13]:
prompt=ChatPromptTemplate.from_template(template)

In [14]:
chain=(
    {"context":reteriver,"question":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [15]:
question1 = "Which 1994 animated movie has a rating of 8.5?"

In [16]:
text_reply=chain.invoke(question1)

In [17]:
text_reply

' (a) Toy Story (b) Jurassic Park (c) Jurassic Park (d) Jurassic Park (e) Jurassic Park (f) Jurassic Park\n\nExplanation:\n\nSolution:\n\nJurassic Park is the best choice as it has a rating of 8.5.\n\nNote: The rating of the movies is not mentioned in the question.'

In [18]:
print(wrap_text(text_reply))

 (a) Toy Story (b) Jurassic Park (c) Jurassic Park (d) Jurassic Park (e) Jurassic Park (f)
Jurassic Park

Explanation:

Solution:

Jurassic Park is the best choice as it has a rating of 8.5.

Note: The rating of the movies is not mentioned in the question.


# **Self Query retriever**

A self-query retriever is a retrieval system that can analyze a natural language question and use it to query itself. Here's how it works:

User Input: You provide a question in plain English. Understanding the Question: The retriever uses a large language model (LLM) to understand the intent and meaning behind your question.

Building a Structured Query: The LLM then translates your question into a structured query that a search engine can understand. This structured query might include keywords and filters based on the details you provided in your question.

Searching the Datastore: The retriever uses the structured query to search its underlying datastore, which is typically a vector store.

Returning Results: The retriever retrieves the documents from the datastore that are most relevant to your question.

We use metadata-filtering to filter out the important chunks.

When it can be used: It will be effective where you have to search in a small subset of the large document. Suppose you want to know about a particular department type like “Sales” in the whole document. Then you need to add this metadata info of department type in each chunk. And filter accordingly.

Overall, self-query retrieval is a powerful technique that leverages the capabilities of LLMs to achieve a more sophisticated and user-centric approach to information retrieval within LangChain models.



In [19]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [20]:
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [21]:
document_content_description = "Brief summary of a movie"

In [22]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt
)

In [23]:

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)

In [24]:
prompt

FewShotPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Swift\\"), eq(\\"artist\\", \\"Katy Perry\\")), lt(\\"length\\", 180), eq(\\"genre\\", \\"pop\\"))"\n}}\n```'}, {'i': 2, 'data_source': '```json\n{{\n    "con

In [25]:
!pip install lark

In [26]:
import lark

In [27]:

output_parser = StructuredQueryOutputParser.from_components()

In [28]:
Query_constructor=prompt | llm | output_parser

In [29]:
print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [31]:
Query_constructor.invoke(
    {
        "query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"
    }
)

StructuredQuery(query=' ', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='year', value=1990), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Luc Besson'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='rating', value=7.0)]), limit=None)

In [33]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=Query_constructor,
    vectorstore=vectorstores,
    structured_query_translator=ChromaTranslator(),
)

In [34]:
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[]

In [35]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

In [36]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [38]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

text_reply = chain.invoke("Tell me about the movie which have rating more than 7.")

print(wrap_text(text_reply))

KeyError: "Input to FewShotPromptTemplate is missing variables {'query'}.  Expected: ['query'] Received: ['context', 'question']\nNote: if you intended {query} to be part of the string and not a variable, please escape it with double curly braces like: '{{query}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [42]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

# ... (other code) ...

# Define a RunnableLambda to extract the question and pass it as 'query'
question_extractor = RunnableLambda(lambda x: {"query": x["question"]})

# Update the chain to include the question_extractor
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | question_extractor  # Extract the question and rename it to 'query'
    | prompt
    | llm
    | StrOutputParser()
)

text_reply = chain.invoke("Tell me about the movie which have rating more than 7.")

print(wrap_text(text_reply))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


OutputParserException: Parsing text
```json
{
    "query": "",
    "filter": "rating(gt(7))"
}
```


<< Example 4. >>
Data Source:
```json
{
    "content": "Lyrics of a song",
    "attributes": {
    "artist": {
        "type": "string",
        "description": "Name of the song artist"
    },
    "length": {
        "type": "integer",
        "description": "Length of the song in seconds"
    },
    "genre": {
        "type": "string",
        "description": "The song genre, one of "pop", "rock" or "rap""
    }
}
}
```

User Query:
What are songs that were not published on Spotify

Structured Request:
```json
{
    "query": "",
    "filter": "NO_FILTER"
}
```


<< Example 5. >>
Data Source:
```json
{
    "content": "Brief summary of a movie",
    "attributes": {
    "genre": {
        "description": "The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        "type": "string"
    },
    "year": {
        "description": "T

In [ ]:
text_reply = chain.invoke("Tell me about the movie which have rating more than 7.")

print(wrap_text(text_reply))